In [3]:
# OPENCV
import cv2 as cv
import numpy as np
from PIL import ImageGrab
from functools import partial
import pyautogui

ImageGrab.grab = partial(ImageGrab.grab, all_screens=True)

In [4]:
def hsvColorSlider():
    cv.namedWindow('HSV Color Slider')
    cv.createTrackbar('Hue Min', 'HSV Color Slider', 0, 179, nothing)
    cv.createTrackbar('Hue Max', 'HSV Color Slider', 104, 179, nothing)
    cv.createTrackbar('Sat Min', 'HSV Color Slider', 0, 255, nothing)
    cv.createTrackbar('Sat Max', 'HSV Color Slider', 255, 255, nothing)
    cv.createTrackbar('Val Min', 'HSV Color Slider', 124, 255, nothing)
    cv.createTrackbar('Val Max', 'HSV Color Slider', 255, 255, nothing)

def nothing(x):
    pass

def getHSV():
    hue_min = cv.getTrackbarPos('Hue Min', 'HSV Color Slider')
    hue_max = cv.getTrackbarPos('Hue Max', 'HSV Color Slider')
    sat_min = cv.getTrackbarPos('Sat Min', 'HSV Color Slider')
    sat_max = cv.getTrackbarPos('Sat Max', 'HSV Color Slider')
    val_min = cv.getTrackbarPos('Val Min', 'HSV Color Slider')
    val_max = cv.getTrackbarPos('Val Max', 'HSV Color Slider')
    return hue_min, hue_max, sat_min, sat_max, val_min, val_max

# hue_min = 0
# hue_max = 179
# sat_min = 0
# sat_max = 255
# val_min = 124
# val_max = 255

In [5]:
def convert_to_binary(image):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    # remove background
    (hmin, hmax, smin, smax, vmin, vmax) = getHSV()
    lower = np.array([hmin, smin, vmin])
    upper = np.array([hmax, smax, vmax])
    mask = cv.inRange(hsv, lower, upper)

    krn = cv.getStructuringElement(cv.MORPH_RECT, (50, 30))
    dlt = cv.dilate(mask, krn, iterations=5)
    res = 255 - cv.bitwise_and(dlt, mask)
    # # Apply threshold
    # _, res = cv.threshold(hsv, 0, 255, cv.THRESH_OTSU)
    return res

In [6]:
# detect checkers board
def detect_checkers_board(img):
    # convert to gray scale
    mask = convert_to_binary(img)
    # mask = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # find the chess board corners
    ret, corners = cv.findChessboardCorners(mask, (7, 7), None)
    # if found, draw corners
    if ret == True:
        # draw corners
        cv.drawChessboardCorners(img, (7, 7), corners, ret)
        return True, corners
    return False, None

In [7]:
# get the board corners
def get_board_corners(corners):
    # get the corners
    corners = corners.reshape(-1, 2)
    x_delta = abs(corners[0][0] - corners[1][0])
    y_delta = abs(corners[0][1] - corners[7][1])

    # get the top left corner
    top_left = corners[0][0] - x_delta, corners[0][1] - y_delta
    # get the top right corner
    top_right = corners[6][0] + x_delta, corners[6][1] - y_delta
    # get the bottom left corner
    bottom_left = corners[42][0] - x_delta, corners[42][1] + y_delta
    # get the bottom right corner
    bottom_right = corners[48][0] + x_delta, corners[48][1] + y_delta
    return top_left, top_right, bottom_left, bottom_right

In [8]:
def get_board_squares(squareCoords, corners):
    # get the corners
    corners = corners.reshape(-1, 2)
    x_delta = abs(corners[0][0] - corners[1][0])
    y_delta = abs(corners[0][1] - corners[7][1])
    
    home_coords = corners[0][0] - x_delta/2, corners[0][1] - y_delta/2

    x = home_coords[0] + ((x_delta) * squareCoords[1])
    y = home_coords[1] + ((y_delta) * squareCoords[0])
    coords = (x, y)

    return coords, x_delta, y_delta

In [9]:
def crop_board(img, corners):
    # get the corners
    top_left, top_right, bottom_left, bottom_right = get_board_corners(corners)
    # get the width of the board
    width = int(np.sqrt((top_right[0] - top_left[0])**2 + (top_right[1] - top_left[1])**2))
    # get the height of the board
    height = int(np.sqrt((top_left[0] - bottom_left[0])**2 + (top_left[1] - bottom_left[1])**2))
    # get the perspective transform matrix
    pts1 = np.float32([top_left, top_right, bottom_left, bottom_right])
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    matrix = cv.getPerspectiveTransform(pts1, pts2)
    # apply the perspective transform
    result = cv.warpPerspective(img, matrix, (width, height))
    return result

In [10]:
def detect_pieces(boardimg, corners):
    if corners is not None:
        # get the corners
        top_left, _, _, bottom_right = get_board_corners(corners)
        # detect white squares
        # convert to gray scale
        gray = cv.cvtColor(boardimg, cv.COLOR_BGR2GRAY)
        # apply threshold
        _, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
        # find contours
        contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        # get the squares
        # apply only the squares that are inside the board
        pieces = []
        for cnt in contours:
            x, y, w, h = cv.boundingRect(cnt)
            if bottom_right[0] > x > top_left[0] and bottom_right[1] > y > top_left[1]:
                if 25 < w < 50 and 25 < h < 50:
                    pieces.append((x, y, w, h))
    return pieces

In [11]:
def match_images(img, template):
    # convert to gray scale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    # get the template size
    w, h = template.shape[::-1]
    # match the template
    res = cv.matchTemplate(gray, template, cv.TM_CCOEFF_NORMED)
    # get the threshold
    threshold = 0.8
    loc = np.where(res >= threshold), w, h

    return loc

In [12]:
def getMove(prevBoard, newBoard):
    direction = [[1, 1], [1, -1], [-1, 1], [-1, -1]]
    # get the index of the changed squares
    changed = np.where(prevBoard != newBoard)
    # get the number of changed squares
    if changed is not None:
        changed = list(zip(changed[0], changed[1]))
        if len(changed) == 2:
            # get the selected square
            selected = (changed[0][0], changed[0][1])
            # get the moved square
            move = (changed[1][0], changed[1][1])
            # get the move direction
            move_direction = [selected[0] - move[0], selected[1] - move[1]]
            return selected, move
    
    else:
        return None, None

In [13]:
originalBoard = np.array([[0, 1, 0, 1, 0, 1, 0, 1],
                            [1, 0, 1, 0, 1, 0, 1, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 1, 0, 1, 0, 1, 0, 1],
                            [1, 0, 1, 0, 1, 0, 1, 0]])

testBoard = np.array([[0, 1, 0, 1, 0, 1, 0, 1],
                        [1, 0, 1, 0, 1, 0, 1, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0],
                        [0, 1, 0, 1, 0, 1, 0, 0],
                        [1, 0, 1, 0, 1, 0, 1, 0]])

selected, move = getMove(originalBoard, testBoard)
selected, move

((5, 6), (6, 7))

In [14]:
def update_board_state(boardimg, corners):
    pieces = detect_pieces(boardimg, corners)
    sorted_pieces = []
    board = np.zeros((8, 8), dtype=int)
    for row in range(8):
        for col in range(8):
            square = (row, col)
            coords, x_delta, y_delta = get_board_squares(square, corners)
            for idx, piece in enumerate(pieces):
                x, y, _, _ = piece
                if coords[0] - x_delta/2 < x < coords[0] + x_delta/2 and coords[1] - y_delta/2 < y < coords[1] + y_delta/2:
                    board[row][col] = 1
                    sorted_pieces.append(piece, square, coords)
                    print(idx, piece, coords, square)
                    break
    return board, sorted_pieces

In [15]:
# only screen 1
screen = ImageGrab.grab(all_screens=True)
print(f"Resolution: {screen.size}, Mode: {screen.mode}, Format: {screen.format}")

# screen right : (1920, 0, 3840, 1080)
# screen left : (0, 0, 1920, 1080)

screen = screen.crop((1920, 0, 3840, 1080))
screen = np.array(screen)

Resolution: (3840, 1080), Mode: RGB, Format: None


In [16]:
def initGame(numGames):
    turn = 'b'
    # initialize the board
    prevBoard = np.array([[0, 1, 0, 1, 0, 1, 0, 1],
                          [1, 0, 1, 0, 1, 0, 1, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 1, 0, 1, 0, 1, 0, 1],
                          [1, 0, 1, 0, 1, 0, 1, 0]])
    
    bot = 'b' if numGames % 2 == 0 else 'w'
    our = 'w' if numGames % 2 == 0 else 'b'
    return turn, prevBoard, bot, our


In [24]:
import numpy as np
testBoard = np.array([[0, 1, 0, 1, 0, 1, 0, 1],
                    [1, 0, 1, 0, 1, 0, 1, 0],
                    [0, 3, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 2, 0, 2, 0, 2, 0, 2],
                    [2, 0, 2, 0, 2, 0, 2, 0]])

print("     0 1 2 3 4 5 6 7")
print("    -----------------")
for idx, row in enumerate(testBoard):
    print(f"{idx} | {row}")

print(np.where(testBoard == 3)[0][0], np.where(testBoard == 3)[1][0])

     0 1 2 3 4 5 6 7
    -----------------
0 | [0 1 0 1 0 1 0 1]
1 | [1 0 1 0 1 0 1 0]
2 | [0 3 0 0 0 0 0 0]
3 | [0 0 0 0 0 0 0 0]
4 | [0 0 0 0 0 0 0 0]
5 | [0 0 0 0 0 0 0 0]
6 | [0 2 0 2 0 2 0 2]
7 | [2 0 2 0 2 0 2 0]
2 1


In [17]:
flipBoard = np.flip(testBoard, 0)
flipBoard = np.flip(flipBoard, 1)
for row in flipBoard:
    print(row)
print(np.where(flipBoard == 3)[0][0], np.where(flipBoard == 3)[1][0])

[0 2 0 2 0 2 0 2]
[2 0 2 0 2 0 2 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 3 0]
[0 1 0 1 0 1 0 1]
[1 0 1 0 1 0 1 0]
5 6


In [18]:
pickerwindowBG = np.zeros((300, 512, 3), np.uint8)
# hsvColorSlider()
# (hmin, hmax, smin, smax, vmin, vmax) = getHSV()
# cv.imshow('HSV Color Slider', pickerwindowBG)

hue_min = 0
hue_max = 179
sat_min = 0
sat_max = 255
val_min = 124
val_max = 255

numGames = 0
startedFlag = False

while numGames < 50:
    if startedFlag == False:
        print("Game started")
        startedFlag = True
        turn, prevBoard, bot, our = initGame(numGames)

    print(bot, our, numGames)

    screen = ImageGrab.grab(all_screens=True)
    # print(f"Resolution: {screen.size}, Mode: {screen.mode}, Format: {screen.format}")

    # screen right : (1920, 0, 3840, 1080)
    # screen left : (0, 0, 1920, 1080)

    screen = screen.crop((1920, 0, 3840, 1080))
    screen = np.array(screen)

    screen_binary = convert_to_binary(screen)
    ret, corners = detect_checkers_board(screen)
    screen = cv.cvtColor(screen, cv.COLOR_BGR2RGB)

    if ret:
        print("Checkers board detected")
        screen_binary = cv.cvtColor(screen_binary, cv.COLOR_GRAY2RGB)

        # get the pieces
        pieces = detect_pieces(screen_binary, corners)
        print(len(pieces), pieces)
        for piece in pieces:
            x, y, w, h = piece
            cv.rectangle(screen_binary, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # update board state
        newBoard, sorted_pieces = update_board_state(screen_binary, corners)
        if turn == bot:
            selected, move = getMove(prevBoard, newBoard)

        screen_binary = crop_board(screen_binary, corners)
        
        cv.imshow('screen_binary', screen_binary)
    else:
        print("Checkers board not detected")
        # check game end
        endimg = cv.imread('end.png')
        matchinfo = match_images(screen, endimg)
        loc, w, h = matchinfo
        if loc[0].size > 0:
            print("Game ended")
            for pt in zip(*loc[::-1]):
                cv.rectangle(screen, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
            
            # reset the board
            pyautogui.moveTo(loc[1][-1] + 200, loc[0][-1] + 310)
            pyautogui.click()
            numGames += 1
            startedFlag = False

        else:
            print("Game not ended")
    # show screen
    cv.imshow('screen', screen)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cv.destroyAllWindows()

Game started
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board not detected
Game ended
Game started
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board not detected
Game ended
Game started
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board not detected
Game ended
Game started
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board not detected
Game ended
Game started
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board not detected
Game ended
Game started
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board not detected
Game ended
Game started
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board not detected
Game ended
Game started
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board not detected
Game ended
Game started
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board not detected
Game ended
Game started
Resolution: (3840, 1080), Mode: RGB, Format: None
Checkers board not 

In [ ]:
# get the board corners
if corners is not None:
    top_left, top_right, bottom_left, bottom_right = get_board_corners(corners)
    print(f"Top left: {top_left}, Top right: {top_right}, Bottom left: {bottom_left}, Bottom right: {bottom_right}")
    coords = get_board_squares((1, 0), corners)
    print(coords)

Top left: (716.0, 268.93216), Top right: (1222.5, 268.93216), Bottom left: (716.0, 774.5679), Bottom right: (1222.5, 774.5679)
(747.75, 363.53392028808594)


In [ ]:
if corners is not None:
    for i in range(8):
        for j in range(8):
            coords, x_delta, y_delta = get_board_squares((i, j), corners)
            print(f"Square {i}, {j}: {coords}, {x_delta}, {y_delta}")
            # pyautogui.moveTo(coords)

Square 0, 0: (747.75, 300.46607971191406), 63.5, 63.067840576171875
Square 0, 1: (811.25, 300.46607971191406), 63.5, 63.067840576171875
Square 0, 2: (874.75, 300.46607971191406), 63.5, 63.067840576171875
Square 0, 3: (938.25, 300.46607971191406), 63.5, 63.067840576171875
Square 0, 4: (1001.75, 300.46607971191406), 63.5, 63.067840576171875
Square 0, 5: (1065.25, 300.46607971191406), 63.5, 63.067840576171875
Square 0, 6: (1128.75, 300.46607971191406), 63.5, 63.067840576171875
Square 0, 7: (1192.25, 300.46607971191406), 63.5, 63.067840576171875
Square 1, 0: (747.75, 363.53392028808594), 63.5, 63.067840576171875
Square 1, 1: (811.25, 363.53392028808594), 63.5, 63.067840576171875
Square 1, 2: (874.75, 363.53392028808594), 63.5, 63.067840576171875
Square 1, 3: (938.25, 363.53392028808594), 63.5, 63.067840576171875
Square 1, 4: (1001.75, 363.53392028808594), 63.5, 63.067840576171875
Square 1, 5: (1065.25, 363.53392028808594), 63.5, 63.067840576171875
Square 1, 6: (1128.75, 363.53392028808594)

In [ ]:
list = pieces.copy()
print(len(list))
# list = sorted(list, key=lambda x: x[0])
list = sorted(list, key=lambda x: x[0])
print(list)
print(list[10])

16
[(728, 343, 41, 42), (791, 280, 42, 42), (854, 343, 42, 42), (917, 280, 42, 42), (981, 343, 41, 42), (1044, 280, 42, 42), (1107, 343, 42, 42), (1170, 280, 42, 42), (730, 725, 37, 33), (793, 664, 36, 33), (920, 662, 35, 33), (858, 725, 34, 34), (983, 725, 37, 33), (1046, 664, 36, 33), (1173, 662, 35, 33), (1111, 725, 34, 34)]
(920, 662, 35, 33)


In [ ]:
pyautogui.moveTo(list[8][0], list[8][1])

In [ ]:
# register the pieces to the board matrix
board = np.zeros((8, 8), dtype=int)
sorted_pieces = []
for row in range(8):
    for col in range(8):
        square = (row, col)
        coords, x_delta, y_delta = get_board_squares(square, corners)
        for idx, piece in enumerate(pieces):
            x, y, _, _ = piece
            if coords[0] - x_delta/2 < x < coords[0] + x_delta/2 and coords[1] - y_delta/2 < y < coords[1] + y_delta/2:
                board[row][col] = 1
                sorted_pieces.append(piece, square, coords)
                print(idx, piece, coords, square)
                break
                # pyautogui.moveTo((coords[0] - x_delta/2, coords[1] - y_delta/2), duration=0.25)
                # pyautogui.moveTo((coords[0] + x_delta/2, coords[1] + y_delta/2), duration=0.25)


for row in board:
    print(row)

15 (791, 280, 42, 42) (811.25, 300.46607971191406) (0, 1)
14 (917, 280, 42, 42) (938.25, 300.46607971191406) (0, 3)
13 (1044, 280, 42, 42) (1065.25, 300.46607971191406) (0, 5)
12 (1170, 280, 42, 42) (1192.25, 300.46607971191406) (0, 7)
11 (728, 343, 41, 42) (747.75, 363.53392028808594) (1, 0)
10 (854, 343, 42, 42) (874.75, 363.53392028808594) (1, 2)
9 (981, 343, 41, 42) (1001.75, 363.53392028808594) (1, 4)
8 (1107, 343, 42, 42) (1128.75, 363.53392028808594) (1, 6)
5 (793, 664, 36, 33) (811.25, 678.8731231689453) (6, 1)
7 (920, 662, 35, 33) (938.25, 678.8731231689453) (6, 3)
4 (1046, 664, 36, 33) (1065.25, 678.8731231689453) (6, 5)
6 (1173, 662, 35, 33) (1192.25, 678.8731231689453) (6, 7)
3 (730, 725, 37, 33) (747.75, 741.9409637451172) (7, 0)
2 (858, 725, 34, 34) (874.75, 741.9409637451172) (7, 2)
1 (983, 725, 37, 33) (1001.75, 741.9409637451172) (7, 4)
0 (1111, 725, 34, 34) (1128.75, 741.9409637451172) (7, 6)
[0 1 0 1 0 1 0 1]
[1 0 1 0 1 0 1 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0

[(791, 280, 42, 42),
 (917, 280, 42, 42),
 (1044, 280, 42, 42),
 (1170, 280, 42, 42),
 (728, 343, 41, 42),
 (854, 343, 42, 42),
 (981, 343, 41, 42),
 (1107, 343, 42, 42),
 (793, 664, 36, 33),
 (920, 662, 35, 33),
 (1046, 664, 36, 33),
 (1173, 662, 35, 33),
 (730, 725, 37, 33),
 (858, 725, 34, 34),
 (983, 725, 37, 33),
 (1111, 725, 34, 34)]

In [ ]:
newcoords = []
for coords in list:
    x, y, w, h = coords
    pyautogui.moveTo(x + w/2, y + h/2)
    newcoords.append((x + w/2, y + h/2))
print(newcoords)

[(784.0, 310.0), (910.0, 310.0), (1037.0, 310.0), (1163.0, 310.0), (720.5, 373.0), (973.5, 373.0), (1100.0, 373.0), (910.0, 436.5), (1099.5, 624.0), (909.5, 687.5), (783.0, 689.5), (1036.0, 689.5), (720.5, 750.5), (847.0, 751.0), (973.5, 750.5), (1100.0, 751.0)]
